# 2D Quantum Simulation
## The Mathematical Details

In this simulation, we want to solve the two-dimensional time-dependent Schrodinger equation in the presence of electromagnetic fields. For simplicity, we will limit the magnetic field to one that is given by a uniform field in the z direction: $\boldsymbol{B}(\boldsymbol{x}) = B \hat{z}$, but will entertain an arbitrary electric scalar potential $V(\boldsymbol{x}, t)$. For future reference, it should be noted that the vector potential for this field configuration is given by $\boldsymbol{A}(\boldsymbol{x}) = -\frac{1}{2} \boldsymbol{x} \times \boldsymbol{B} = \frac{B}{2} (x \hat{y} - y \hat{x})$.

Our first step is to write out the Schrodinger equation in the presence of electromagnetic fields:
$$
\frac{1}{2m} \left(\hat{\boldsymbol{p}} - q\boldsymbol{A}\right)^2 \psi + V\psi = \hat{E} \psi
$$

We make the substitutions $\hat{\boldsymbol{p}} \rightarrow -i \nabla$, $\hat{E} \rightarrow i \partial_t$, and expand the momentum operator:
$$
\frac{1}{2m} \left(-\nabla^2 + iq\nabla\cdot\boldsymbol{A} + iq\boldsymbol{A}\cdot\nabla + q^2A^2\right)^2 \psi + V \psi = i\partial_t\psi
$$

If we assume the Coulomb gauge ($\nabla\cdot\boldsymbol{A} = 0$) and substitute our expression for the vector potential, this becomes
$$
\frac{1}{2m} \left(-(\partial_x^2\psi + \partial_y^2\psi) + \frac{iqB}{2}\left(x\partial_y\psi - y\partial_x\psi\right) + \frac{q^2B^2}{4}(x^2+y^2)\psi \right) + V\psi = i\partial_t\psi
$$

If we further split up the wave function into real/imaginary parts $\psi = R + iI$, then we get two PDE's (one for each component):
$$
\begin{align}
\partial_t R &= \frac{1}{2m} \left(-(\partial_x^2 I + \partial_y^2 I) + \frac{qB}{2}(x\partial_y R - y\partial_x R) + \frac{q^2B^2}{4}(x^2+y^2)I\right) + VI \\
\partial_t I &= \frac{1}{2m} \left((\partial_x^2 R + \partial_y^2 R) + \frac{qB}{2} (x\partial_y I - y\partial_x I) - \frac{q^2B^2}{4}(x^2 + y^2)R\right) - VR
\end{align}
$$

It is then a simple matter to discretize these equations and solve for the values of the wave function at the next time step. We shall consider two different discretization schemes; first the Forward Time Centered Space (FTCS):
$$
\partial_t X = (X_{t+1} - X_t)/dt,\qquad \partial_x X = (X_{i+1} - X{i-1})/2\,dx,\qquad \partial_x^2 X = (X_{i+1} - 2X_{i} + X_{i-1})/dx^2;
$$
which is quite simple to implement, but is unfortunately numerically unstable. If we instead use a Centered Time Centered Space (CTCS) scheme:
$$
\partial_t X = (X_{t+1} - X_{t-1})/2\,dt,\qquad \partial_x X = (X_{i+1} - X{i-1})/2\,dx,\qquad \partial_x^2 X = (X_{i+1} - 2X_{i} + X_{i-1})/dx^2;
$$
the stability increases greatly (it is not unconditionally stable, but should be stable for $dt/2m\,dx^2 \ll 1$), but at the small cost of having to keep around data from the previous timestep. In practice, we will use both; the FTCS scheme will be used on the very first time step (since the Schrodinger equation is first order, we only want to specify the initial timestep, and not its derivative), and any further updates will use the CTCS step.